In [ ]:
import jax.numpy as jnp
import jax
from jax import experimental
from jax.experimental import optimizers
from jax import grad, jit, vmap
from jax import random
from scipy.special import jv
import os
import datetime
import time
import sys

import numpy as np
import pandas as pd
import time
import math
from jax import device_put

# Manuscript: "Target Acoustic Field and Transducer State Optimization using Diff-PAT"
# Authors: Tatsuki Fushimi, Kenta Yamamoto, Yoichi Ochiai
# Corresponding email: tfushimi@slis.tsukuba.ac.jp
# JAX version 0.2.17

In [ ]:
print(jax.__version__)

0.2.17


In [ ]:
import os
if not os.path.exists('results'):
    os.makedirs('results')

In [ ]:
# Setup Transducer Array
N=16 # number of transducer assuming square array
pitch=0.0105 # pitch of transducer array
size=[N,N] 

# Generate transducer array of arbitary size N.
tra_x = np.array([ (i % N - (N//2-0.5))*pitch for i in range(N * N)]).reshape(N, N)
tra_y = tra_x.T
tra_z = np.full((N,N), 0.0)

#Transducer Settings
P0=1.0 # Transducer Amplitude at 1 m
l_ambda = 346.0 / 40000.0 # wavelength
k = 2.0*math.pi/l_ambda # wavenumber
r0 = 0.005 # Radius of transducer

# MAX number of iteration for the optimizer
dropping_threshold = 1000; 

# Preamble for optimizer specfication
settings_t = [1,2,3]
transducer_side = ['A', 'B','C']
target_side = ['i','ii','iii']
N_list = [2, 4]

# Export transducer array if needed. 
if False:
  np.savetxt('transducer_x.csv', np.array( [tra_x.T.ravel()] ).ravel(), delimiter=',')
  np.savetxt('transducer_y.csv', np.array( [tra_y.T.ravel()] ).ravel(), delimiter=',')
  np.savetxt('transducer_z.csv', np.array( [tra_z.T.ravel()] ).ravel(), delimiter=',')

In [ ]:
# Pre-calculating propagation parts. This part is invariant during optimization, as long as the focus point does not change. 
def prop_matrix(array_x, array_y, array_z, x_p, y_p, z_p, P0, k, r0):
  prop = []
  for xi in range(point_num):
    dist_map = jnp.sqrt( jnp.power((array_x - x_p[xi]), 2) + jnp.power((array_y - y_p[xi]), 2) + jnp.power((array_z - z_p[xi]), 2) )

    sin_alpha_map = jnp.sqrt( (jnp.power((array_x - x_p[xi]), 2) + jnp.power((array_y - y_p[xi]), 2)) ) / dist_map
    sin_alpha_map = jnp.where(sin_alpha_map == 0, jax.numpy.finfo(jnp.float64).tiny, sin_alpha_map)

    amplitude_map = ( 2 * jv(1, k*r0*sin_alpha_map) * P0 / (k*r0*sin_alpha_map*dist_map) )
    prop.append( jax.lax.complex( amplitude_map * jnp.cos(k*dist_map), amplitude_map * jnp.sin(k*dist_map) ) )
  
  prop = device_put(jnp.array(prop))

  return prop

In [ ]:
# Three types of propagation mode, A - Phase, B - Amplitude, C - Phase and Amplitude
@jit
def Propagation_A(Tr):
  part_A = Tr[0]
  trans_amplitude = jax.lax.complex(jnp.ones(size), jnp.zeros(size))
  trans_phase_copmlex = jax.lax.complex(jnp.cos(part_A), jnp.sin(part_A))
  return trans_amplitude, trans_phase_copmlex

@jit
def Propagation_B(Tr):
  part_A = Tr[0]
  part_A = jnp.abs(part_A)
  part_A = part_A / jnp.max(part_A)
  trans_amplitude = jax.lax.complex(part_A, jnp.zeros(size))
  trans_phase_copmlex = jax.lax.complex(jnp.ones(size), jnp.zeros(size))
  return trans_amplitude, trans_phase_copmlex

@jit
def Propagation_C(Tr):
  part_A = Tr[0]
  part_B = Tr[1]

  tr_set = jax.lax.complex(part_A, part_B)
  temp_amp = jnp.abs(tr_set);
  temp_amp = temp_amp / jnp.max(temp_amp)
    
  trans_amplitude = jax.lax.complex(temp_amp, jnp.zeros(size))
  trans_phase = jnp.angle(tr_set)
  trans_phase_copmlex = jax.lax.complex(jnp.cos(trans_phase), jnp.sin(trans_phase))
  return trans_amplitude, trans_phase_copmlex

# Calculate Acoustic Pressure Field at the target point, output amplitude and phase. 
@jit 
def calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i):
  point_Re = jnp.sum(jnp.real(trans_phase_copmlex) * jnp.real(trans_amplitude*transducer_prop[i]) - jnp.imag(trans_phase_copmlex) * jnp.imag(trans_amplitude*transducer_prop[i]))
  point_Im = jnp.sum(jnp.real(trans_phase_copmlex) * jnp.imag(trans_amplitude*transducer_prop[i]) + jnp.imag(trans_phase_copmlex) * jnp.real(trans_amplitude*transducer_prop[i]))
  point_amp = jnp.sqrt( point_Re ** 2 + point_Im ** 2 )
  point_phase = jnp.angle(jax.lax.complex(point_Re, point_Im))
  return point_amp, point_phase

# Loss function depending on the target point state. i - Phase, ii - Amplitude, iii - Phase and Amplitude
@jit
def ErrFunc_i(point_phase, target_pha, i):
  err = (jnp.cos(point_phase) - jnp.cos(target_pha[i]))**2 + (jnp.sin(point_phase) - jnp.sin(target_pha[i]))**2
  return err

@jit
def ErrFunc_ii(point_amp, target_amp, i):
  err = (point_amp - target_amp[i])**2
  return err

@jit
def ErrFunc_iii(point_amp, point_phase, target_amp, target_pha, i):
  err = (point_amp*jnp.cos(point_phase) - target_amp[i]*jnp.cos(target_pha[i]))**2 + (point_amp*jnp.sin(point_phase) - target_amp[i]*jnp.sin(target_pha[i]))**2
  return err

From here, each combination of optimizer functions are defined. This is not ideal programming, but JAX does not prefer conditionals during automatic differentiation. 

In [ ]:
@jit
def loss_func_Ai(Tr, transducer_prop, target_pha):
    trans_amplitude, trans_phase_copmlex = Propagation_A(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_i(point_phase, target_pha, i)
      loss += (1/point_num)*(err)
    return loss

@jit
def step_Ai(step, opt_state, transducer_prop, target_pha):
  value, grads = jax.value_and_grad(loss_func_Ai)(get_params(opt_state), transducer_prop, target_pha)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Aii(Tr, transducer_prop, target_amp):
    trans_amplitude, trans_phase_copmlex = Propagation_A(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_ii(point_amp, target_amp, i)
      loss += (1/point_num)*(err)
    return loss
@jit
def step_Aii(step, opt_state, transducer_prop, target_amp):
  value, grads = jax.value_and_grad(loss_func_Aii)(get_params(opt_state), transducer_prop, target_amp)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Aiii(Tr, transducer_prop, target_amp, target_pha):
    trans_amplitude, trans_phase_copmlex = Propagation_A(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_iii(point_amp, point_phase, target_amp, target_pha, i)
      loss += (1/point_num)*(err)
    return loss

@jit
def step_Aiii(step, opt_state, transducer_prop, target_amp, target_pha):
  value, grads = jax.value_and_grad(loss_func_Aiii)(get_params(opt_state), transducer_prop, target_amp, target_pha)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Bi(Tr, transducer_prop, target_pha):
    trans_amplitude, trans_phase_copmlex = Propagation_B(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_i(point_phase, target_pha, i)
      loss += (1/point_num)*(err)
    return loss
@jit
def step_Bi(step, opt_state, transducer_prop, target_pha):
  value, grads = jax.value_and_grad(loss_func_Bi)(get_params(opt_state), transducer_prop, target_pha)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Bii(Tr, transducer_prop, target_amp):
    trans_amplitude, trans_phase_copmlex = Propagation_B(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_ii(point_amp, target_amp, i)
      loss += (1/point_num)*(err)
    return loss

@jit
def step_Bii(step, opt_state, transducer_prop, target_amp):
  value, grads = jax.value_and_grad(loss_func_Bii)(get_params(opt_state), transducer_prop, target_amp)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Biii(Tr, transducer_prop, target_amp, target_pha):
    trans_amplitude, trans_phase_copmlex = Propagation_B(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_iii(point_amp, point_phase, target_amp, target_pha, i)
      loss += (1/point_num)*(err)
    return loss
@jit
def step_Biii(step, opt_state, transducer_prop, target_amp, target_pha):
  value, grads = jax.value_and_grad(loss_func_Biii)(get_params(opt_state), transducer_prop, target_amp, target_pha)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Ci(Tr, transducer_prop, target_pha):
    trans_amplitude, trans_phase_copmlex = Propagation_C(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_i(point_phase, target_pha, i)
      loss += (1/point_num)*(err)
    return loss
@jit
def step_Ci(step, opt_state, transducer_prop, target_pha):
  value, grads = jax.value_and_grad(loss_func_Ci)(get_params(opt_state), transducer_prop, target_pha)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Cii(Tr, transducer_prop, target_amp):
    trans_amplitude, trans_phase_copmlex = Propagation_C(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_ii(point_amp, target_amp, i)
      loss += (1/point_num)*(err)
    return loss
    
@jit
def step_Cii(step, opt_state, transducer_prop, target_amp):
  value, grads = jax.value_and_grad(loss_func_Cii)(get_params(opt_state), transducer_prop, target_amp)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

In [ ]:
@jit
def loss_func_Ciii(Tr, transducer_prop, target_amp, target_pha):
    trans_amplitude, trans_phase_copmlex = Propagation_C(Tr)
    loss = 0
    # Evaluating Loss Function
    for i in range(point_num):
      point_amp, point_phase = calculate_pressure(trans_amplitude, trans_phase_copmlex, transducer_prop, i)

      err = ErrFunc_iii(point_amp, point_phase, target_amp, target_pha, i)
      loss += (1/point_num)*(err)
    return loss

@jit
def step_Ciii(step, opt_state, transducer_prop, target_amp, target_pha):
  value, grads = jax.value_and_grad(loss_func_Ciii)(get_params(opt_state), transducer_prop, target_amp, target_pha)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

Main loop for the optimizer. 

In [ ]:
# Main Loop
for nn in range(len(N_list)): 
  # Load Targets
  point_num = N_list[nn]
  x_p_list = pd.read_csv('Target_X_N_' + str(point_num) + '.csv', header=None)
  x_p_list = x_p_list.values
  y_p_list = pd.read_csv('Target_Y_N_' + str(point_num) + '.csv', header=None)
  y_p_list = y_p_list.values
  z_p_list = pd.read_csv('Target_Z_N_' + str(point_num) + '.csv', header=None)
  z_p_list = z_p_list.values

  number_of_sample = x_p_list.shape[1]

  target_amp_list = pd.read_csv('Target_Amp_N_' + str(point_num) + '.csv', header=None)
  target_amp_list = target_amp_list.values

  target_pha_list = pd.read_csv('Target_Phase_N_' + str(point_num) + '.csv', header=None)
  target_pha_list = target_pha_list.values

  for ii in range(len(settings_t)):
    for jj in range(len(settings_t)):
      rng = np.random.default_rng(2021) # For reproducibility
      transducer_side_settings = settings_t[ii] # 1 for Phase, 2 for Amplitude, 3 for amplitude and phase
      target_plane_settings = settings_t[jj] # 1 for Phase, 2 for Amplitude, 3 for amplitude and phase
      loss_all = []
      amplitude_all = []
      phase_all = []
      for kk in range(1000): #number of samples
        x_p = x_p_list[:, kk]
        y_p = y_p_list[:, kk]
        z_p = z_p_list[:, kk]
        
        transducer_prop = prop_matrix(tra_x, tra_y, tra_z, x_p, y_p, z_p, P0, k, r0)

        # Initialize tau
        target_amp=target_amp_list[:, kk]
        target_pha=target_pha_list[:, kk]

        Tr_A = rng.random((size[0], size[1]))
        if transducer_side_settings == 1:
          Tr_A = rng.random((size[0], size[1])) 

        if transducer_side_settings == 3:
          Tr_B = rng.random((size[0], size[1]))
          Tr = [Tr_A, Tr_B]
        else:
          Tr = [Tr_A]

        # Initialize Optimizer
        opt_init, opt_update, get_params = jax.experimental.optimizers.adam(0.1, b1=0.9, b2=0.999, eps=1e-08)
        opt_state = opt_init(Tr)
        loss_list = []
        for st in range(dropping_threshold): # Bit messy to list the combination in if-loop, but JAX did not like conditionals passed on. 
          # Making static arguments inputs may improve the situation... 
          if transducer_side_settings == 1:
            if target_plane_settings == 1:
              value, opt_state = step_Ai(st, opt_state, transducer_prop, target_pha)
            elif target_plane_settings == 2:
              value, opt_state = step_Aii(st, opt_state, transducer_prop, target_amp)
            elif target_plane_settings == 3:
              value, opt_state = step_Aiii(st, opt_state, transducer_prop, target_amp, target_pha)
          elif transducer_side_settings == 2:
            if target_plane_settings == 1:
              value, opt_state = step_Bi(st, opt_state, transducer_prop, target_pha)
            elif target_plane_settings == 2:
              value, opt_state = step_Bii(st, opt_state, transducer_prop, target_amp)
            elif target_plane_settings == 3:
              value, opt_state = step_Biii(st, opt_state, transducer_prop, target_amp, target_pha)
          elif transducer_side_settings == 3:
            if target_plane_settings == 1:
              value, opt_state = step_Ci(st, opt_state, transducer_prop, target_pha)
            elif target_plane_settings == 2:
              value, opt_state = step_Cii(st, opt_state, transducer_prop, target_amp)
            elif target_plane_settings == 3:
              value, opt_state = step_Ciii(st, opt_state, transducer_prop, target_amp, target_pha)
          loss_list.append(value)
          
        # Exporting Data
        Tr = get_params(opt_state)    
        if transducer_side_settings == 1:
          part_A = Tr[0]
          amplitude = jnp.ones(size)
          phase = part_A
        elif transducer_side_settings == 2:
          part_A = Tr[0]
          part_A = jnp.abs(part_A)
          part_A = part_A / jnp.max(part_A)
          amplitude = jnp.abs(part_A)
          phase = jnp.zeros(size)
        elif transducer_side_settings == 3:
          part_A = Tr[0]
          part_B = Tr[1]
          tr_set = jax.lax.complex(part_A, part_B)
          temp_amp = jnp.abs(tr_set);
          temp_amp = temp_amp / jnp.max(temp_amp) 
          amplitude = temp_amp
          phase = jnp.angle(tr_set)

        amplitude_exports = np.array(amplitude)
        phase_exports = np.array(phase)

        loss_all.append(np.array(loss_list))
        amplitude_all.append(np.array(amplitude_exports.T.ravel()).ravel())
        phase_all.append(np.array(phase_exports.T.ravel()).ravel())

      phase_1d_arr = np.array( phase_all )
      amp_1d_arr = np.array( amplitude_all )
      loss_1d_arr = np.array( loss_all )
      np.savetxt('results/Loss_exports_N_' + str(point_num) + '_Trans_' + transducer_side[ii] + '_Target_'+ target_side[jj] + '_settings.csv', loss_1d_arr, delimiter=',')
      np.savetxt('results/Phase_exports_N_' + str(point_num) + '_Trans_' + transducer_side[ii] + '_Target_'+ target_side[jj] + '_settings.csv', phase_1d_arr, delimiter=',')
      np.savetxt('results/Amplitude_exports_N_' + str(point_num) + '_Trans_' + transducer_side[ii] + '_Target_'+ target_side[jj] + '_settings.csv', amp_1d_arr, delimiter=',')
      print('----- N_' + str(point_num) + '_Trans_' + transducer_side[ii] + '_Target_'+ target_side[jj] + ' COMPLETED -----')

----- N_2_Trans_A_Target_i COMPLETED -----
----- N_2_Trans_A_Target_ii COMPLETED -----
----- N_2_Trans_A_Target_iii COMPLETED -----
----- N_2_Trans_B_Target_i COMPLETED -----
----- N_2_Trans_B_Target_ii COMPLETED -----
----- N_2_Trans_B_Target_iii COMPLETED -----
----- N_2_Trans_C_Target_i COMPLETED -----
----- N_2_Trans_C_Target_ii COMPLETED -----
----- N_2_Trans_C_Target_iii COMPLETED -----
----- N_4_Trans_A_Target_i COMPLETED -----
----- N_4_Trans_A_Target_ii COMPLETED -----
----- N_4_Trans_A_Target_iii COMPLETED -----
----- N_4_Trans_B_Target_i COMPLETED -----
----- N_4_Trans_B_Target_ii COMPLETED -----
----- N_4_Trans_B_Target_iii COMPLETED -----
----- N_4_Trans_C_Target_i COMPLETED -----
----- N_4_Trans_C_Target_ii COMPLETED -----
----- N_4_Trans_C_Target_iii COMPLETED -----


In [ ]:
!zip -r /content/colab_results.zip /content/results/

  adding: content/results/ (stored 0%)
  adding: content/results/Phase_exports_N_4_Trans_A_Target_i_settings.csv (deflated 59%)
  adding: content/results/Loss_exports_N_2_Trans_A_Target_i_settings.csv (deflated 85%)
  adding: content/results/Amplitude_exports_N_2_Trans_C_Target_iii_settings.csv (deflated 61%)
  adding: content/results/Amplitude_exports_N_2_Trans_B_Target_i_settings.csv (deflated 59%)
  adding: content/results/Loss_exports_N_4_Trans_B_Target_i_settings.csv (deflated 81%)
  adding: content/results/Phase_exports_N_4_Trans_C_Target_ii_settings.csv (deflated 60%)
  adding: content/results/Loss_exports_N_2_Trans_B_Target_ii_settings.csv (deflated 80%)
  adding: content/results/Loss_exports_N_4_Trans_B_Target_ii_settings.csv (deflated 77%)
  adding: content/results/Phase_exports_N_2_Trans_B_Target_ii_settings.csv (deflated 100%)
  adding: content/results/Phase_exports_N_2_Trans_C_Target_iii_settings.csv (deflated 61%)
  adding: content/results/Phase_exports_N_2_Trans_A_Target